<div class="alert alert-block alert-success">
<h1>Thin Flate Plate</h1>
</div>

<p style="font-size:17px; color:black; text-align:justify"></p>

<p style="font-size:17px; color:black; text-align:justify"></p> 

<p style="font-size:17px; color:red; text-align:justify">The plots is interactive.</p> 

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# HIDDEN
# General Purpose
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy.integrate import odeint
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from sympy import *
from IPython.display import display
import nbinteract as nbi

# Jupyter Specifics
from IPython.display import HTML
#from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, FloatSlider, Layout


%matplotlib inline
matplotlib.rcParams['xtick.direction'] = 'out'
matplotlib.rcParams['ytick.direction'] = 'out'

style = {'description_width': '150px'}
slider_layout = Layout(width='90%')

In [3]:
# HIDDEN

alphaDefault = 12
cnDefault = 1.2
caDefault = 0.03

style = {'description_width': 'initial'}
sliderLayout = Layout(width='80%')
boxLayout = Layout(width='70%')
c = 1  # chord length



x = Symbol('x')
integrtionRange = (x, 0, c)

class Givens(object):
    lowerPressure = '2*10**4*(x-1)**2 + 1.73*10**5'   # lower level pressure
    upperPressure = '4*10**4*(x-1)**2 + 5.4*10**4'  # upper level pressure
    lowerShear = '731*x**(-0.2)' # lower level shear stress
    upperShear = '288*x**(-0.2)' # upper level shear stress
    
    # Normal force
    @staticmethod
    def N():
        return float(integrate(Givens.lowerPressure + "-(" + Givens.upperPressure + ")", integrtionRange))


    # Axial force
    @staticmethod
    def A():
        return float(integrate(Givens.lowerShear + "+(" + Givens.upperShear + ")", integrtionRange))


    # Lift
    @staticmethod
    def L(a):
        a = a * np.pi / 180  # convert angle to degrees
        Ap = Givens.A()
        Np = Givens.N()
        return float(Np*np.cos(a) - Ap*np.sin(a))


    # Lift coefficient
    @staticmethod
    def Cl(a, ca,cn):
        a = a * np.pi / 180  # convert angle to degree
        return float(cn*np.cos(a) - ca*np.sin(a))


    # Drag
    @staticmethod
    def D(a):
        a = a * np.pi / 180  # convert angle to degrees
        Ap = Givens.A()
        Np = Givens.N()
        return float(Ap*np.cos(a) + Np*np.sin(a))

    # drag coefficient
    @staticmethod
    def Cd(a, ca, cn):
        a = a * np.pi / 180  # convert angle to degrees
        return float(ca*np.cos(a) + cn*np.sin(a))


    # Moment at the leading edge
    @staticmethod
    def Mle():
        return float(integrate("(" + Givens.upperPressure + "-(" + Givens.lowerPressure + "))*x", integrtionRange))


    # Moment at the quarter chord
    @staticmethod
    def Mc4(a):
        a = a * np.pi / 180  # convert angle to degrees
        return float(Givens.Mle() + Givens.L(a)*(c/4))


    # Locaion of center of pressure
    @staticmethod
    def Xcp(a):
        a = a * np.pi / 180  # convert angle to degrees
        return float(-Givens.Mle()/Givens.N())


pl_widget = widgets.Text(
    value=Givens.lowerPressure,
    placeholder='Enter an expression',
    description=r'$p_l$',
    disabled=False
)

pu_widget = widgets.Text(
    value=Givens.upperPressure,
    placeholder='Enter an expression',
    description=r'$p_u$',
    disabled=False
)

tl_widget = widgets.Text(
    value=Givens.lowerShear,
    placeholder='Enter an expression',
    description=r'$\tau_l$',
    disabled=False
)

tu_widget = widgets.Text(
    value=Givens.lowerShear,
    placeholder='Enter an expression',
    continuous_update=False,
    description=r'$\tau_u$',
    disabled=False
)


def update_pl(*args):
    pl_widget.value = pl_widget.value
    Givens.lowerPressure = pl_widget.value 
pl_widget.observe(update_pl, 'value')


def update_pu(*args):
    pu_widget.value = pu_widget.value
    Givens.upperPressure = pu_widget.value   
pu_widget.observe(update_pu, 'value')


def update_tl(*args):
    tl_widget.value = tl_widget.value
    Givens.lowerShear = tl_widget.value    
tl_widget.observe(update_tl, 'value')


def update_tu(*args):
    tu_widget.value = tu_widget.value
    Givens.upperShear = tu_widget.value
tu_widget.observe(update_tu, 'value')




# Angle of attack widget settings
alpha_widget = widgets.FloatSlider(
    value=alphaDefault,
    min=0,
    max=15.0,
    step=0.5,
    description=r'Angle of Attack, $\alpha$, (unit: $^{\circ}$):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=sliderLayout,
    style=style
)





# Normal Force widget settings
NormalForce = widgets.FloatText(
    value=Givens.N(),
    description=r"Normal Force per unit span, $N'$, (unit: $N$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_normal(*args):
    NormalForce.value = Givens.N()
alpha_widget.observe(update_normal, 'value')



# Axial Force widget settings
AxialForce = widgets.FloatText(
    value=Givens.A(),
    description=r"Axial Force per unit span, $A'$, (unit: $N$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_axial(*args):
    AxialForce.value = Givens.A()
alpha_widget.observe(update_axial, 'value')




# Lift widget settings
Lift = widgets.FloatText(
    value=Givens.L(alphaDefault),
    description=r"Lift per unit span, $L'$, (unit: $N$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_lift(*args):
    Lift.value = Givens.L(alpha_widget.value)
alpha_widget.observe(update_lift, 'value')




# Drag widget settings
Drag = widgets.FloatText(
    value=Givens.D(alphaDefault),
    description=r"Drag per unit span, $D'$, (unit: $N$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_drag(*args):
    Drag.value = Givens.D(alpha_widget.value)
alpha_widget.observe(update_drag, 'value')





# Moment at the leading edge widget settings
MomLE = widgets.FloatText(
    value=Givens.Mle(),
    description=r"Moment at the Leading Edge per unit span, $M_{LE}'$, (unit: $N\cdot m$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_mle(*args):
    MomLE.value = Givens.Mle()
alpha_widget.observe(update_mle, 'value')




# Moment at the Quarter Chord widget settings
MomC4 = widgets.FloatText(
    value=Givens.Mc4(alphaDefault),
    description=r"Moment at the Quarter Chord per unit span, $M_{c/4}'$, (unit: $N\cdot m$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_mc4(*args):
    MomC4.value = Givens.Mc4(alpha_widget.value)
alpha_widget.observe(update_mc4, 'value')




# Location of Center of Pressure widget settings
LocCoP = widgets.FloatText(
    value=Givens.Xcp(alphaDefault),
    description=r"Center of Pressure Location (unit: $m$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_xcop(*args):
    LocCoP.value = Givens.Xcp(alpha_widget.value)
alpha_widget.observe(update_xcop, 'value')

In [4]:
# HIDDEN
display(pl_widget)
display(pu_widget)
display(tl_widget)
display(tu_widget)

display(alpha_widget)
display(NormalForce)
display(AxialForce)
display(Lift)
display(Drag)
display(MomLE)
display(MomC4)
display(LocCoP)

Text(value='2*10**4*(x-1)**2 + 1.73*10**5', description='$p_l$', placeholder='Enter an expression')

Text(value='4*10**4*(x-1)**2 + 5.4*10**4', description='$p_u$', placeholder='Enter an expression')

Text(value='731*x**(-0.2)', description='$\\tau_l$', placeholder='Enter an expression')

Text(value='731*x**(-0.2)', continuous_update=False, description='$\\tau_u$', placeholder='Enter an expression…

FloatSlider(value=12.0, continuous_update=False, description='Angle of Attack, $\\alpha$, (unit: $^{\\circ}$):…

...

FloatText(value=109613.7529662517, description="Lift per unit span, $L'$, (unit: $N$):", layout=Layout(width='…

FloatText(value=24601.32877496298, description="Drag per unit span, $D'$, (unit: $N$):", layout=Layout(width='…

FloatText(value=-57833.333333333336, description="Moment at the Leading Edge per unit span, $M_{LE}'$, (unit: …

FloatText(value=-29751.35164147804, description="Moment at the Quarter Chord per unit span, $M_{c/4}'$, (unit:…

FloatText(value=0.5148367952522256, description='Center of Pressure Location (unit: $m$):', layout=Layout(widt…

In [5]:
# HIDDEN
alphaDefault = 12
cnDefault = 1.2
caDefault = 0.03


# Angle of attack widget settings
alpha_widget2 = widgets.FloatSlider(
    value=alphaDefault,
    min=0,
    max=15.0,
    step=0.5,
    description=r'Angle of Attack, $\alpha$, (unit: $^{\circ}$):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=sliderLayout,
    style=style
)



# Axial force coeefficient widget settings
ca_widget = widgets.FloatSlider(
    value=caDefault,
    min=-1,
    max=1,
    step=0.01,
    description=r'Axial Force Coefficient, ($c_a$):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=sliderLayout,
    style=style
)


# Normal force coeefficient widget settings
cn_widget = widgets.FloatSlider(
    value=cnDefault,
    min=-3,
    max=3,
    step=0.1,
    description=r'Norma Force Coefficient ($c_n$):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=sliderLayout,
    style=style
)






# Lift coeficient widget settings
LiftCE = widgets.FloatText(
    value=Givens.Cl(alphaDefault, caDefault, cnDefault),
    description=r"Lift Coefficient ($c_l$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
    layout=boxLayout,
    style=style
)

def update_liftce(*args):
    LiftCE.value = Givens.Cl(alpha_widget2.value, ca_widget.value, cn_widget.value)
    
ca_widget.observe(update_liftce, 'value')
alpha_widget2.observe(update_liftce, 'value')
cn_widget.observe(update_liftce, 'value')


# Lift coeficient widget settings
DragCE = widgets.FloatText(
    value=Givens.Cd(alphaDefault, caDefault, cnDefault),
    description=r"Drag Coefficient ($c_d$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
    layout=boxLayout,
    style=style
)

def update_dragce(*args):
    DragCE.value = Givens.Cd(alpha_widget2.value, ca_widget.value, cn_widget.value)
    
ca_widget.observe(update_dragce, 'value')
alpha_widget2.observe(update_dragce, 'value')
cn_widget.observe(update_dragce, 'value')



In [6]:
# HIDDEN
display(alpha_widget2)
display(ca_widget)
display(cn_widget)
display(LiftCE)
display(DragCE)


FloatSlider(value=12.0, continuous_update=False, description='Angle of Attack, $\\alpha$, (unit: $^{\\circ}$):…

FloatSlider(value=0.03, continuous_update=False, description='Axial Force Coefficient, ($c_a$):', layout=Layou…

FloatSlider(value=1.2, continuous_update=False, description='Norma Force Coefficient ($c_n$):', layout=Layout(…

FloatText(value=1.167539770156034, description='Lift Coefficient ($c_l$):', layout=Layout(width='70%'), style=…

FloatText(value=0.2788384570033253, description='Drag Coefficient ($c_d$):', layout=Layout(width='70%'), style…